In [2]:
from constants_dat_file import *
#from gemini_implementation import *
from IPython.display import clear_output
import matplotlib.pyplot as plt
import nbtlib

world_name = 'maps_imgs'
image_name = '11.jpg'
image_path = f"images/{image_name}"
image_extention = image_path.split('.')[1]
out_name = f'{image_path.split('/')[1].replace('.'+image_extention,'')}'

# MAP CREATION

In [3]:
def load_image_for_map(image_path, channels: int, mirror_horizontal: bool = False):
    image_bytes = tf.io.read_file(image_path)
    image_tensor = tf.image.decode_image(image_bytes, channels=channels) 

    if mirror_horizontal:
        image_tensor = tf.image.flip_left_right(image_tensor)

    int_image = tf.cast(image_tensor, dtype=tf.int32)
    return int_image

In [4]:
def prepare_image_size_to_map(image_tensor):
    current_height = image_tensor.shape[0]
    current_width = image_tensor.shape[1] 

    num_rows_of_maps = tf.math.ceil(current_height / 128).numpy()
    num_cols_of_maps = tf.math.ceil(current_width / 128).numpy()

    new_height = int(128 * num_rows_of_maps)
    new_width = int(128 * num_cols_of_maps)

    resized_image = tf.image.resize(image_tensor, ((new_height, new_width)))
    resized_image = tf.cast(resized_image, dtype=tf.int32)
    print(f'RESIZE = {num_rows_of_maps} rows, {num_cols_of_maps} columns')
    return resized_image, int(num_rows_of_maps), int(num_cols_of_maps)

In [5]:
def cut_image_into_128_squares(original_image, num_rows_of_maps, num_cols_of_maps):
    image_squares = []
    row_splits = tf.split(original_image, num_or_size_splits=num_rows_of_maps, axis=0)

    for row_strip in row_splits:
        col_splits = tf.split(row_strip, num_or_size_splits=num_cols_of_maps, axis=1)
        for map_square in col_splits:
            image_squares.append(map_square)

    return image_squares

In [6]:
def prepare_square_for_prediction(sliced_image):
    to_predict = tf.reshape(sliced_image, [128*128, 3])
    return to_predict

def get_img_slices_for_prediction(image_squares, num_rows_of_maps, num_cols_of_maps):
    for_predictions = []
    for one_square in image_squares:
        one_square_for_prediction = prepare_square_for_prediction(one_square)
        for_predictions.append(one_square_for_prediction)
    return for_predictions

In [7]:
def rgb_to_ints(for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps):
    colors_square_ids = []
    color_model, mcid_to_rgb = get_mcid_color_model()
    for square in for_predictions:
        distances , inds = color_model.kneighbors(square)
        colors_square_ids.append(inds+4)
    return colors_square_ids

In [8]:
def create_template_datfile(title="some_data_idk"):
    root = nbtlib.File()
    root.name = title

    data_compound = nbtlib.Compound()
    data_compound['scale'] = nbtlib.Byte(0)
    data_compound['dimension'] = nbtlib.String("minecraft:overworld")
    data_compound['trackingPosition'] = nbtlib.Byte(0)
    data_compound['locked'] = nbtlib.Byte(1)
    data_compound['width'] = nbtlib.Short(128)
    data_compound['height'] = nbtlib.Short(128)
    
    data_compound['xCenter'] = nbtlib.Int(0)
    data_compound['zCenter'] = nbtlib.Int(0)

    data_compound['colors'] = nbtlib.ByteArray([5] * (128 * 128))

    root['data'] = data_compound
    return root

In [9]:
def get_current_map(world_name,parent):
    world_data_path = os.path.join(parent, f'saves/{world_name}/data')
    map_files = os.listdir(world_data_path)
    map_files = [x for x in map_files if x.startswith('map_')]
    map_files = [x.replace('map_','').replace('.dat','') for x in map_files]
    map_files = [int(x) for x in map_files]
    if len(map_files) == 0:
        return 0
    current_map = max(map_files) + 1
    return current_map
def create_maps(maps_colors, num_rows_of_maps, num_cols_of_maps, world_name):
    current_directory = os.getcwd()
    parent = os.path.dirname(current_directory)
    base_image_name = os.path.splitext(os.path.basename(image_path))[0]
    n_file = get_current_map(world_name,parent)
    init_map = n_file

    print(f'Generating {len(maps_colors)} new maps.')

    for ind, map_file_data in enumerate(maps_colors):
        row = ind // num_cols_of_maps
        col = ind % num_cols_of_maps
        x_center = col * 128 + 64
        z_center = row * 128 + 64

        # output_dir = os.path.join(current_directory, "maps", base_image_name)
        # if not os.path.exists(output_dir):
        #     os.makedirs(output_dir)
        
        #output_filename_local = os.path.join(output_dir, f'map_{n_file}.dat')
        world_data_path = os.path.join(parent, f'saves/{world_name}/data')

        output_filename_world = os.path.join(world_data_path, f'map_{n_file}.dat')
        nbtfile = create_template_datfile(title=f"{base_image_name}_map_{n_file}")

        color_ids = [entry[0] for entry in map_file_data]
        map_byte_data = bytearray(color_ids) 

        nbtfile["data"]["colors"] = nbtlib.ByteArray(map_byte_data)
        nbtfile["data"]["xCenter"] = nbtlib.Int(x_center)
        nbtfile["data"]["zCenter"] = nbtlib.Int(z_center)

        try:
            #nbtfile.save(output_filename_local) 
            nbtfile.save(output_filename_world) 
        except Exception as e:
            print(f"Error saving map_{n_file}.dat: {e}")

        n_file += 1

    final_map = n_file
    print(f"Map generation complete. Generated maps from ID {init_map} to {final_map - 1}.")
    return init_map, final_map, (num_rows_of_maps, num_cols_of_maps)
def check_world_folder_existence(world_name):
    current_directory = os.getcwd()
    parent = os.path.dirname(current_directory)
    world_directory = os.path.join(parent, 'saves',world_name,'data')
    return os.path.exists(world_directory)

In [10]:
def map_pipeline(image_path, channels: int, mirror_horizontal: bool = False, world_name: str = None):
    
    image_tensor = load_image_for_map(image_path, channels = 3, mirror_horizontal = False)
    original_image, num_rows_of_maps, num_cols_of_maps = prepare_image_size_to_map(image_tensor)
    image_squares = cut_image_into_128_squares(original_image, num_rows_of_maps, num_cols_of_maps)
    for_predictions = get_img_slices_for_prediction(image_squares, num_rows_of_maps, num_cols_of_maps)
    maps_colors = rgb_to_ints(for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps)
    init_map, final_map, (num_rows_of_maps, num_cols_of_maps) = create_maps(maps_colors, num_rows_of_maps, num_cols_of_maps, world_name)
    
    return init_map, final_map, (num_rows_of_maps, num_cols_of_maps)

# MCFUNCTION

In [11]:
def fill_line(x1,y1,z1,x2,y2,z2,material):
    return f"fill ~{x1} ~{y1} ~{z1} ~{x2} ~{y2} ~{z2} minecraft:{material}\n"

def summon_map_line(x_offset,y_offset,z_offset,facing,n_map, rotation = 0):
    return f"summon minecraft:glow_item_frame ~-{x_offset} ~{y_offset} ~{z_offset} " \
           f"{{ItemRotation:{rotation},Facing:{facing},Item:{{id:\"minecraft:filled_map\",count:1,components:{{\"minecraft:map_id\":{n_map}}}}}}}\n"

In [12]:
def create_map_support(num_rows_of_maps, num_cols_of_maps, map_range,facing):
    mcfunction_lines = []
    if facing =='south':
        mcfunction_lines.append(
                        fill_line(
                            0, 
                            0, 
                            1, 
                            -(num_cols_of_maps - 1), 
                            num_rows_of_maps - 1, 
                            1,
                            'dirt'
                            )
                        )
    if facing =='north':
        mcfunction_lines.append(
                        fill_line(
                            -1, 
                            0, 
                            1, 
                            -(num_cols_of_maps)+1, 
                            num_rows_of_maps - 1, 
                            1,
                            'dirt'
                            )
                        )   
    if facing in 'west':
        mcfunction_lines.append(
                        fill_line(
                            -1, 
                            0, 
                            1, 
                            -1, 
                            num_rows_of_maps - 1, 
                            num_cols_of_maps,
                            'dirt'
                            )
                        )
    if facing in 'east':
        mcfunction_lines.append(
                        fill_line(
                            1, 
                            0, 
                            1, 
                            1, 
                            num_rows_of_maps - 1, 
                            num_cols_of_maps,
                            'dirt'
                            )
                        )
    if facing in ['up','down']:
        mcfunction_lines.append(
                        fill_line(
                            0, 
                            -1, 
                            0, 
                            -(num_rows_of_maps - 1), 
                            -1, 
                            num_cols_of_maps -1,
                            'dirt'
                            )
                        )
    first_map = map_range[0]
    last_map = map_range[1]
    maps_numbers = [int(x.replace('map_','').replace('.dat','')) for x in os.listdir('maps') if x.endswith('.dat')]
    maps_numbers = [x for x in maps_numbers if x in range(first_map,last_map)]
    maps_numbers = sorted(maps_numbers)

    return mcfunction_lines, maps_numbers

In [13]:
def write_frame_wall(num_rows_of_maps, num_cols_of_maps, map_range, out_name,world_name):
    current_directory = os.getcwd()
    parent = os.path.dirname(current_directory)
    mcfunction_dir = f'saves/{world_name}/datapacks/img/data/print/function/'
    
    for facing_direction in ['south','north','east','west','up','down']:
        
        mcfunction_lines, maps_numbers = create_map_support(num_rows_of_maps,
                                                                        num_cols_of_maps,
                                                                        map_range, facing_direction)
        for row_idx in range(num_rows_of_maps):
            for col_idx in range(num_cols_of_maps):
                current_map_id = map_range[0] + (row_idx * num_cols_of_maps) + col_idx
                if facing_direction == 'south':
                    mcfunction_lines.append(
                        summon_map_line(
                            col_idx, 
                            (num_rows_of_maps - 1 - row_idx),
                            facing_side[facing_direction+'_z'],
                            facing_side[facing_direction],
                            current_map_id))
                if facing_direction == 'north':
                    mcfunction_lines.append(
                        summon_map_line(
                            (num_cols_of_maps - col_idx), 
                            (num_rows_of_maps - 1 - row_idx),
                            facing_side[facing_direction+'_z'],
                            facing_side[facing_direction],
                            current_map_id))
                if facing_direction == 'east':
                    mcfunction_lines.append(
                        summon_map_line(
                            0, 
                            (num_rows_of_maps - 1 - row_idx),
                            col_idx+1,
                            facing_side[facing_direction],
                            current_map_id))
                if facing_direction == 'west':
                    mcfunction_lines.append(
                        summon_map_line(
                            0, 
                            (num_rows_of_maps - 1 - row_idx),
                            (num_cols_of_maps - 1 - col_idx)+1,
                            facing_side[facing_direction],
                            current_map_id))
                if facing_direction == 'up':
                    mcfunction_lines.append(
                        summon_map_line(
                            row_idx, 
                            0,
                            col_idx,
                            facing_side[facing_direction],
                            current_map_id,
                            rotation = 1
                        ))
                if facing_direction == 'down':
                    mcfunction_lines.append(
                        summon_map_line(
                            row_idx, 
                            -2,
                            (num_cols_of_maps - 1 - col_idx),
                            facing_side[facing_direction],
                            current_map_id,
                            rotation = 1
                        ))
            
               
            with open(os.path.join(parent,
                f'{mcfunction_dir}{out_name}_{facing_direction}.mcfunction'), "w") as f:
                for line in mcfunction_lines:
                    f.write(line)
        else: pass

# LIVE PIPELINE

In [15]:
init_map, final_map, map_range = map_pipeline(image_path,
                        channels = 3,
                        mirror_horizontal = False,
                        world_name = 'maps_imgs')

RESIZE = 6.0 rows, 13.0 columns
Generating 78 new maps.
Map generation complete. Generated maps from ID 312 to 389.


In [16]:
write_frame_wall(num_rows_of_maps,
                 num_cols_of_maps,
                 map_range,
                 out_name,
                 world_name)

In [61]:
init_map

"Minecraft_World: popo doesn't exists"

In [26]:
init_map

False

In [44]:
current_directory = os.getcwd()
parent = os.path.dirname(current_directory)

In [45]:
get_current_map(current_directory,parent)

78

In [17]:
current_map

78

In [ ]:
(num_rows_of_maps - 1 - row_idx)

In [4]:
init_map, final_map, block_size = create_maps(image_path,world_name= world_name,channels = 3)

I0000 00:00:1749153876.205756    4580 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1749153876.210120    4580 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6717 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:03:00.0, compute capability: 6.1


RESIZE = 6.0 rows, 13.0 columns
Generating 78 new maps.
Map generation complete. Generated maps from ID 0 to 77.


In [4]:
write_frame_wall(num_rows_of_maps = 6,
                 num_cols_of_maps = 13,
                 map_range =(0,78),
                 out_name = out_name,
                 world_name = world_name)

In [ ]:
create_map_support(12,6,(0,77),south)

In [ ]:
e = create_template_datfile(title="some_data_idk")

In [ ]:
e

In [ ]:
nbtfile = create_template_datfile(title=f"{image_name}_map_{0}.dat")

In [ ]:
type(nbtfile)

In [ ]:
block_size = (2,4)
write_frame_wall(num_rows_of_maps = block_size[0],
                 num_cols_of_maps =block_size[1],
                 map_range = (init_map,final_map),
                 out_name = out_name)

In [ ]:
init_map

In [ ]:
# import numpy as np
# from sklearn.neighbors import NearestNeighbors
# import joblib
# import tensorflow as tf
# import os


# def get_mcid_color_model():
#     mcid_to_rgb = joblib.load('color_dicts/mcid_to_rgb.pkl')
    
#     colors = np.array(list(mcid_to_rgb.values()))
#     color_model = NearestNeighbors(n_neighbors=1, metric='euclidean')
#     color_model.fit(colors)
#     return color_model, mcid_to_rgb

# def load_image_for_map(image_path, channels: int):
#     image_bytes = tf.io.read_file(image_path)
#     # Use decode_image for broader support, or ensure it's always JPEG
#     image_tensor = tf.image.decode_image(image_bytes, channels=channels) 
#     int_image = tf.cast(image_tensor, dtype=tf.int32)
#     return int_image

# def prepare_image_size_to_map(image_path, channels: int):
#     image_tensor = load_image_for_map(image_path, channels = channels)
#     current_height = image_tensor.shape[0] # Height
#     current_width = image_tensor.shape[1]  # Width

#     # Calculate number of map chunks needed for height and width
#     num_rows_of_maps = tf.math.ceil(current_height / 128).numpy()
#     num_cols_of_maps = tf.math.ceil(current_width / 128).numpy()

#     new_height = int(128 * num_rows_of_maps)
#     new_width = int(128 * num_cols_of_maps)

#     resized_image = tf.image.resize(image_tensor, ((new_height, new_width)))
#     resized_image = tf.cast(resized_image, dtype=tf.int32)
#     print(f'RESIZE = {num_rows_of_maps} rows, {num_cols_of_maps} columns')
#     return resized_image, int(num_rows_of_maps), int(num_cols_of_maps)

# def cut_image_into_128_squares(image_path, channels: int):
#     image_squares = []
#     original_image, num_rows_of_maps, num_cols_of_maps = prepare_image_size_to_map(image_path, channels)
#     row_splits = tf.split(original_image, num_or_size_splits=num_rows_of_maps, axis=0)

#     for row_strip in row_splits:
#         col_splits = tf.split(row_strip, num_or_size_splits=num_cols_of_maps, axis=1)
#         for map_square in col_splits:
#             image_squares.append(map_square)

#     return image_squares, num_rows_of_maps, num_cols_of_maps
 
# def prepare_square_for_prediction(sliced_image):
#     to_predict = tf.reshape(sliced_image, [128*128, 3])
#     return to_predict

# def get_img_slices_for_prediction(image_path, channels = 3):
#     for_predictions = [] 
#     image_squares, num_rows_of_maps, num_cols_of_maps = cut_image_into_128_squares(image_path, channels = 3)
#     for one_square in image_squares:
#         one_square_for_prediction = prepare_square_for_prediction(one_square)
#         for_predictions.append(one_square_for_prediction)
#     return for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps

# def rgb_to_ints(image_path, channels = 3):
#     for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps = get_img_slices_for_prediction(image_path, channels = 3)
#     colors_square_ids = []
#     color_model, mcid_to_rgb = get_mcid_color_model()
#     for square in for_predictions:
#         distances , inds = color_model.kneighbors(square)
#         colors_square_ids.append(inds+4) # Keep `+4` as per your successful fix for colors
#     return colors_square_ids, num_rows_of_maps, num_cols_of_maps

# def create_maps(image_path, channels = 3):
#     maps_colors, num_rows_of_maps, num_cols_of_maps = rgb_to_ints(image_path, channels = 3)
#     n_file = get_current_map() 
#     init_map = get_current_map()
    
#     print('n maps: ',len(maps_colors))
#     for ind, map_file in enumerate(maps_colors):
#         count = 0
#         nbt_file = nbt.NBTFile("example_map.dat","rb") 
#         color_data = [x[0] for x in map_file]
#         for x in range(128):
#             for y in range(128):
#                 mc_id = color_data[count]
#                 assert type(mc_id) == np.int64
#                 nbt_file["data"]["colors"][x+(y*128)] = mc_id
#                 count+=1
#         nbt_file.write_file(f'maps/map_{n_file}.dat')
#         n_file +=1
#     final_map = n_file
#     return init_map, final_map, (num_rows_of_maps, num_cols_of_maps)
    
# def get_current_map():
#     current_map_files = [x for x in os.listdir('maps') if x.endswith('.dat')]
#     available_map_ids = [int(x.replace('map_','').replace('.dat','')) for x in current_map_files]
#     if len(available_map_ids) == 0:
#         return 0
#     else:
#         return max(available_map_ids) + 1

# def fill_line(x1,y1,z1,x2,y2,z2,material):
#     return f"fill ~-{x1} ~{y1} ~{z1} ~-{x2} ~{y2} ~{z2} minecraft:{material}\n"

# def summon_map_line(x_offset,y_offset,z_offset,n_map):
#     # This function summons a glow item frame with a map
#     # Facing:2 means facing North (positive Z)
#     return f"summon minecraft:glow_item_frame ~-{x_offset} ~{y_offset} ~{z_offset} " +\
#             "{Facing:2,Item:{id:'minecraft:filled_map',count:1,components:{'minecraft:map_id':" +\
#             f"{n_map}"+"}}}\n"

# def create_map_support(num_rows_of_maps, num_cols_of_maps, map_range):
#     mcfunction_lines = []
#     mcfunction_lines.append(
#         fill_line(
#             0, 
#             0, 
#             1, 
#             num_cols_of_maps - 1, 
#             num_rows_of_maps - 1, 
#             1, # z2: Same depth
#             'dirt'
#         )
#     )

#     first_map = map_range[0]
#     last_map = map_range[1]
#     maps_numbers = [int(x.replace('map_','').replace('.dat','')) for x in os.listdir('maps') if x.endswith('.dat')]
#     maps_numbers = [x for x in maps_numbers if x in range(first_map,last_map)]
#     maps_numbers = sorted(maps_numbers) # Ensure maps are sorted numerically
    
#     return mcfunction_lines, maps_numbers
    

# def write_frame_wall(num_rows_of_maps, num_cols_of_maps, map_range, out_name):
#     mcfunction_lines, maps_numbers = create_map_support(num_rows_of_maps, num_cols_of_maps, map_range)
    
#     n_map = map_range[0]
#     for row_idx in range(num_rows_of_maps):
#         for col_idx in range(num_cols_of_maps):
#             inverted_col_idx = (num_cols_of_maps - 1 - col_idx)
            
#             mcfunction_lines.append(
#                 summon_map_line(
#                     inverted_col_idx, 
#                     (num_rows_of_maps - 1 - row_idx),
#                     0,
#                     n_map 
#                 )
#             )
#             n_map += 1
#             if n_map > map_range[1]:
#                 break 
#         if n_map > map_range[1]:
#             break

#     with open(out_name, "w") as f:
#         for line in mcfunction_lines:
#             f.write(line)

In [ ]:
init_map

In [ ]:
write_frame_wall(num_rows_of_maps, num_cols_of_maps, map_range, out_name)

In [ ]:
def get_mcid_color_model():
    mcid_to_rgb = joblib.load('color_dicts/mcid_to_rgb.pkl')
    colors = np.array(list(mcid_to_rgb.values()))
    color_model = NearestNeighbors(n_neighbors=1, metric='euclidean')
    color_model.fit(colors)
    return color_model,mcid_to_rgb
def load_image_for_map(image_path, channels: int):
    image_bytes = tf.io.read_file(image_path)
    image_tensor = tf.image.decode_jpeg(image_bytes, channels=channels)
    int_image = tf.cast(image_tensor, dtype=tf.int32)
    return int_image
def prepare_image_size_to_map(image_path, channels: int):
    image_tensor = load_image_for_map(image_path, channels = channels)
    current_x = image_tensor.shape[0]
    current_y = image_tensor.shape[1]
    x_pieces = tf.math.ceil(current_x / 128).numpy()
    y_pieces = tf.math.ceil(current_y / 128).numpy()
    new_x = int(128 * x_pieces)
    new_y = int(128 * y_pieces)
    resized_image = tf.image.resize(image_tensor, ((new_x, new_y)))
    resized_image = tf.cast(resized_image, dtype=tf.int32)
    print(f'RESIZE = {x_pieces}, {y_pieces}')
    return resized_image, int(x_pieces), int(y_pieces)
# def cut_image_into_128_squares_(image_path, channels: int):
#     image_squares = []
#     original_image, x_pieces, y_pieces = prepare_image_size_to_map(image_path, channels)
#     columns = tf.split(original_image, num_or_size_splits=y_pieces, axis=1)
#     for square in columns:
#         squares = tf.split(square, num_or_size_splits=x_pieces, axis=0)
#         for one_square in squares:
#             image_squares.append(one_square)
#     return image_squares, x_pieces, y_pieces

# def cut_image_into_128_squares(image_path, channels: int):
#     image_squares = []
#     original_image, x_pieces, y_pieces = prepare_image_size_to_map(image_path, channels)

#     # Change the splitting order to row-major
#     rows = tf.split(original_image, num_or_size_splits=x_pieces, axis=0) # Split by rows (height)
#     for row_tensor in rows: # row_tensor is a horizontal strip of the image
#         squares = tf.split(row_tensor, num_or_size_splits=y_pieces, axis=1) # Split each row into columns (width)
#         for one_square in squares:
#             image_squares.append(one_square) # Appends A, B, C, D for the 2x2 example
#     return image_squares, x_pieces, y_pieces


def cut_image_into_128_squares(image_path, channels: int):
    image_squares = []
    original_image, num_rows_of_maps, num_cols_of_maps = prepare_image_size_to_map(image_path, channels)

    # Split the image into horizontal strips (rows of 128px height)
    # original_image.shape[0] is height
    row_splits = tf.split(original_image, num_or_size_splits=num_rows_of_maps, axis=0)

    for row_strip in row_splits:
        # For each horizontal strip, split it into 128x128 squares (columns)
        # row_strip.shape[1] is width
        col_splits = tf.split(row_strip, num_or_size_splits=num_cols_of_maps, axis=1)
        for map_square in col_splits:
            image_squares.append(map_square)

    # Ensure the return values match your x_pieces, y_pieces naming convention
    # where x_pieces is typically horizontal (width) and y_pieces is vertical (height)
    # In your case, it seems x_pieces means num_rows_of_maps and y_pieces means num_cols_of_maps
    return image_squares, num_rows_of_maps, num_cols_of_maps # Returning what you called x_pieces, y_pieces

    
def prepare_square_for_prediction(sliced_image):
    to_predict = tf.reshape(sliced_image, [128*128, 3])
    return to_predict
def get_img_slices_for_prediction(image_path, channels = 3):
    for_predictions = [] 
    image_squares, x_pieces, y_pieces = cut_image_into_128_squares(image_path, channels = 3)
    #print(len(image_squares))
    for ind,one_square in enumerate(image_squares):
        #print(ind)
        one_square_for_prediction = prepare_square_for_prediction(one_square)
        for_predictions.append(one_square_for_prediction)
    return for_predictions, image_squares, x_pieces, y_pieces
def rgb_to_ints(image_path, channels = 3):
    for_predictions, image_squares, x_pieces, y_pieces = get_img_slices_for_prediction(image_path, channels = 3)
    colors_square_ids = []
    color_model, mcid_to_rgb = get_mcid_color_model()
    for square in for_predictions:
        #map_colors, colors_to_id = get_color_map_ids()
        distances , inds = color_model.kneighbors(square)
        colors_square_ids.append(inds+4) 
    return colors_square_ids, x_pieces, y_pieces
def create_maps(image_path, channels = 3):
    maps_colors, x_pieces, y_pieces = rgb_to_ints(image_path, channels = 3)
    n_file = get_current_map() 
    init_map = get_current_map()
    
    
    print('n maps: ',len(maps_colors))
    for ind, map_file in enumerate(maps_colors):
        count = 0
        nbt_file = nbt.NBTFile("example_map.dat","rb")
        color_data = [x[0] for x in map_file]
        for x in range(128):
            for y in range(128):
                mc_id = color_data[count]
                assert type(mc_id) == np.int64
                nbt_file["data"]["colors"][x+(y*128)] = mc_id
                count+=1
        nbt_file.write_file(f'maps/map_{n_file}.dat')
        n_file +=1
    final_map = n_file
    return init_map, final_map, (x_pieces, y_pieces)
    
def get_current_map():
    current_map = [x for x in os.listdir('maps') if x.endswith('.dat')]
    available_maps = [int(x.replace('map_','').replace('.dat','')) for x in current_map]
    if len(available_maps) == 0:
        return 0
    else:
        current_map = max([int(x.replace('map_','').replace('.dat','')) for x in current_map])
    return current_map
def fill_line(x1,y1,z1,x2,y2,z2,material):
    return f"fill ~-{x1} ~{y1} ~{z1} ~-{x2} ~{y2} ~{z2} minecraft:{material}\n"
def summon_map_line(x,y,z,n_map):
    return f"summon minecraft:glow_item_frame ~-{x} ~{y} ~{z} " +\
            "{Facing:2,Item:{id:'minecraft:filled_map',count:1,components:{'minecraft:map_id':" +\
            f"{n_map}"+"}}}\n"
def create_map_support(x,y, map_range):
    mcfunction_lines = []

    first_map = map_range[0]
    last_map = map_range[1]
    maps_numbers = [x for x in os.listdir('maps') if x.endswith('.dat')]
    maps_numbers = [x.replace('map_','') for x in maps_numbers]
    maps_numbers = [x.replace('.dat','') for x in maps_numbers]
    maps_numbers = [int(x) for x in maps_numbers if int(x) in range(first_map,last_map)]
    maps_numbers = sorted(maps_numbers)
    for column in range(x):
        mcfunction_lines.append(fill_line(column,0,1,y,x,1,'dirt'))
    return mcfunction_lines, maps_numbers
    
def write_frame_wall(x,y, map_range, out_name):
    mcfunction_lines, maps_numbers = create_map_support(x,y, map_range)
    n_map = map_range[0]
    for column in range(x):
        for row in range(y):
            mcfunction_lines.append(summon_map_line(row,abs(column-x),0,n_map))
            n_map += 1
            if n_map > map_range[1]:
                break
    with open(out_name, "w") as f:
        for line in mcfunction_lines:
            f.write(line)

In [ ]:
init_map, final_map, block_size = create_maps(image_path, channels = 3)

In [ ]:
init_map

In [ ]:
final_map

In [ ]:
block_size

In [ ]:
write_frame_wall(x = block_size[0], y =block_size[1]+1,
                 map_range = (init_map,final_map), out_name = 'try_11.mcfunction')

In [ ]:
maps_path = 'maps/map_1.dat'

In [ ]:
nbt_file = nbt.NBTFile(maps_path)

In [ ]:
nbt_file['data'].__dict__

In [ ]:
nbt_file['data'].__dict__

In [ ]:
nbt_file['data'].tags[1].colors

In [ ]:
help(nbt_file['data'])

In [ ]:
    with open(out_name, "w") as f:
        for line in mcfunction_lines:
            f.write(line)

In [ ]:
lines = []

In [ ]:
def summon_map_line(x,y,z,n_map):
    return f"summon minecraft:glow_item_frame ~-{x} ~{y} ~{z} " +\
            "{Facing:2,Item:{id:'minecraft:filled_map',count:1,components:{'minecraft:map_id':" +\
            f"{n_map}"+"}}"

In [ ]:
summon_map_line(x = 1, y = 2,z = 3, n_map = 6)

In [ ]:
x = 1
y = 1
z = 1
n_map = 1

In [ ]:
summon_line(x,y,z,n_map)

In [ ]:
# import numpy as np
# from sklearn.neighbors import NearestNeighbors
# import joblib
# import tensorflow as tf
# #from PIL import Image # Add PIL for image loading if not already used elsewhere for decode_jpeg
# import os

# # Ensure PIL is installed: pip install Pillow
# # Ensure nbt is installed: pip install nbtlib (or similar, depending on your NBT library)

# def get_mcid_color_model():
#     mcid_to_rgb = joblib.load('color_dicts/mcid_to_rgb.pkl')
#     # It's safer to sort the keys to ensure consistent indexing for NearestNeighbors
#     # This was discussed in the previous turn; if you're using `inds+4`,
#     # it implies a fixed offset, which might work if your mcid_to_rgb.pkl
#     # always loads in that specific order. For robustness, the explicit
#     # index-to-MCID mapping is better, but for now, we'll keep `inds+4` as you confirmed it works.
    
#     # If the `inds+4` works, it means the 0-based index from NearestNeighbors
#     # maps to the actual MCID after adding 4. This implies your `mcid_to_rgb.pkl`
#     # has MCIDs starting from 4, and they are ordered sequentially in the dictionary values.
#     # For example: {4: (R0,G0,B0), 5: (R1,G1,B1), ...}
    
#     colors = np.array(list(mcid_to_rgb.values()))
#     color_model = NearestNeighbors(n_neighbors=1, metric='euclidean')
#     color_model.fit(colors)
#     return color_model, mcid_to_rgb

# def load_image_for_map(image_path, channels: int):
#     image_bytes = tf.io.read_file(image_path)
#     # Use decode_image for broader support, or ensure it's always JPEG
#     image_tensor = tf.image.decode_image(image_bytes, channels=channels) 
#     int_image = tf.cast(image_tensor, dtype=tf.int32)
#     return int_image

# def prepare_image_size_to_map(image_path, channels: int):
#     image_tensor = load_image_for_map(image_path, channels = channels)
#     current_height = image_tensor.shape[0] # Height
#     current_width = image_tensor.shape[1]  # Width

#     # Calculate number of map chunks needed for height and width
#     num_rows_of_maps = tf.math.ceil(current_height / 128).numpy()
#     num_cols_of_maps = tf.math.ceil(current_width / 128).numpy()

#     new_height = int(128 * num_rows_of_maps)
#     new_width = int(128 * num_cols_of_maps)

#     resized_image = tf.image.resize(image_tensor, ((new_height, new_width)))
#     resized_image = tf.cast(resized_image, dtype=tf.int32)
#     print(f'RESIZE = {num_rows_of_maps} rows, {num_cols_of_maps} columns')
#     return resized_image, int(num_rows_of_maps), int(num_cols_of_maps)

# def cut_image_into_128_squares(image_path, channels: int):
#     image_squares = []
#     original_image, num_rows_of_maps, num_cols_of_maps = prepare_image_size_to_map(image_path, channels)

#     # Split the image into horizontal strips (rows of 128px height)
#     # axis=0 is height
#     row_splits = tf.split(original_image, num_or_size_splits=num_rows_of_maps, axis=0)

#     for row_strip in row_splits:
#         # For each horizontal strip, split it into 128x128 squares (columns)
#         # axis=1 is width
#         col_splits = tf.split(row_strip, num_or_size_splits=num_cols_of_maps, axis=1)
#         for map_square in col_splits:
#             image_squares.append(map_square)

#     return image_squares, num_rows_of_maps, num_cols_of_maps
 
# def prepare_square_for_prediction(sliced_image):
#     to_predict = tf.reshape(sliced_image, [128*128, 3])
#     return to_predict

# def get_img_slices_for_prediction(image_path, channels = 3):
#     for_predictions = [] 
#     image_squares, num_rows_of_maps, num_cols_of_maps = cut_image_into_128_squares(image_path, channels = 3)
#     for one_square in image_squares:
#         one_square_for_prediction = prepare_square_for_prediction(one_square)
#         for_predictions.append(one_square_for_prediction)
#     return for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps

# def rgb_to_ints(image_path, channels = 3):
#     for_predictions, image_squares, num_rows_of_maps, num_cols_of_maps = get_img_slices_for_prediction(image_path, channels = 3)
#     colors_square_ids = []
#     color_model, mcid_to_rgb = get_mcid_color_model()
#     for square in for_predictions:
#         distances , inds = color_model.kneighbors(square)
#         colors_square_ids.append(inds+4) # Keep `+4` as per your successful fix for colors
#     return colors_square_ids, num_rows_of_maps, num_cols_of_maps

# def create_maps(image_path, channels = 3):
#     maps_colors, num_rows_of_maps, num_cols_of_maps = rgb_to_ints(image_path, channels = 3)
#     n_file = get_current_map() 
#     init_map = get_current_map()
    
#     print('n maps: ',len(maps_colors))
#     for ind, map_file in enumerate(maps_colors):
#         count = 0
#         # Make sure 'example_map.dat' exists and is a valid NBT map template
#         # Or create a new NBTFile structure if you don't have a template
#         nbt_file = nbt.NBTFile("example_map.dat","rb") 
#         color_data = [x[0] for x in map_file]
#         for x in range(128):
#             for y in range(128):
#                 mc_id = color_data[count]
#                 assert type(mc_id) == np.int64
#                 nbt_file["data"]["colors"][x+(y*128)] = mc_id
#                 count+=1
#         nbt_file.write_file(f'maps/map_{n_file}.dat')
#         n_file +=1
#     final_map = n_file
#     return init_map, final_map, (num_rows_of_maps, num_cols_of_maps)
    
# def get_current_map():
#     current_map_files = [x for x in os.listdir('maps') if x.endswith('.dat')]
#     available_map_ids = [int(x.replace('map_','').replace('.dat','')) for x in current_map_files]
#     if len(available_map_ids) == 0:
#         return 0
#     else:
#         # Find the maximum map ID and return the next one
#         return max(available_map_ids) + 1 # Return the next available map ID

# def fill_line(x1,y1,z1,x2,y2,z2,material):
#     # This function creates a fill command relative to the player's position
#     # ~-{x} means negative X offset, ~{y} means positive Y offset
#     return f"fill ~-{x1} ~{y1} ~{z1} ~-{x2} ~{y2} ~{z2} minecraft:{material}\n"

# def summon_map_line(x_offset,y_offset,z_offset,n_map):
#     # This function summons a glow item frame with a map
#     # Facing:2 means facing North (positive Z)
#     return f"summon minecraft:glow_item_frame ~-{x_offset} ~{y_offset} ~{z_offset} " +\
#             "{Facing:2,Item:{id:'minecraft:filled_map',count:1,components:{'minecraft:map_id':" +\
#             f"{n_map}"+"}}}\n"

# def create_map_support(num_rows_of_maps, num_cols_of_maps, map_range):
#     mcfunction_lines = []

#     # Create a single rectangular dirt wall behind where the maps will be placed.
#     # The wall should cover the extent of the map grid.
#     # X-coordinates for fill_line: from 0 (player's X) to num_cols_of_maps - 1 (most negative X offset)
#     # Y-coordinates for fill_line: from 0 (player's Y) to num_rows_of_maps - 1 (most positive Y offset)
#     # Z-coordinates for fill_line: 1 (behind the player for the wall)

#     mcfunction_lines.append(
#         fill_line(
#             0, # x1: Start at player's X (0 offset)
#             0, # y1: Start at player's Y (0 offset)
#             1, # z1: Depth behind player (for the wall)
#             num_cols_of_maps - 1, # x2: End at this X offset (max horizontal extent)
#             num_rows_of_maps - 1, # y2: End at this Y offset (max vertical extent)
#             1, # z2: Same depth
#             'dirt'
#         )
#     )

#     first_map = map_range[0]
#     last_map = map_range[1]
#     maps_numbers = [int(x.replace('map_','').replace('.dat','')) for x in os.listdir('maps') if x.endswith('.dat')]
#     maps_numbers = [x for x in maps_numbers if x in range(first_map,last_map)]
#     maps_numbers = sorted(maps_numbers) # Ensure maps are sorted numerically
    
#     return mcfunction_lines, maps_numbers
    

# def write_frame_wall(num_rows_of_maps, num_cols_of_maps, map_range, out_name):
#     mcfunction_lines, maps_numbers = create_map_support(num_rows_of_maps, num_cols_of_maps, map_range)
    
#     n_map = map_range[0] # Starts from the first map ID generated (e.g., 0 for map_0.dat)

#     # Iterate through the grid of maps in row-major order (top-to-bottom, left-to-right)
#     # This matches how maps_colors are generated in rgb_to_ints
#     for row_idx in range(num_rows_of_maps): # This is the vertical position on the wall
#         for col_idx in range(num_cols_of_maps): # This is the horizontal position on the wall
#             # The n_map variable increments linearly, corresponding to map_0, map_1, etc.
#             # These maps are already in row-major order due to cut_image_into_128_squares.

#             # Vertical position (y_offset for summon_map_line):
#             # (num_rows_of_maps - 1 - row_idx) correctly inverts Y for display
            
#             # Horizontal position (x_offset for summon_map_line):
#             # Invert the col_idx to display left-to-right (from player's perspective)
#             # If col_idx=0 is top-left in data, it should be at the leftmost position on the wall.
#             # Since ~-{x} puts increasing x to the left, we need to map 0 to largest x, and largest x to 0.
#             inverted_col_idx = (num_cols_of_maps - 1 - col_idx)
            
#             mcfunction_lines.append(
#                 summon_map_line(
#                     inverted_col_idx, # Horizontal offset (x_coord in summon_map_line), now inverted
#                     (num_rows_of_maps - 1 - row_idx), # Vertical offset (y_coord in summon_map_line)
#                     0, # Z-position (on the wall, 0 offset from player)
#                     n_map # Map ID
#                 )
#             )
#             n_map += 1
#             if n_map > map_range[1]:
#                 break # Break inner loop if all maps are placed
#         if n_map > map_range[1]:
#             break # Break outer loop if all maps are placed

#     with open(out_name, "w") as f:
#         for line in mcfunction_lines:
#             f.write(line)